In [ ]:
import numpy as np
from cvxopt import matrix, solvers

class SVM:
    def __init__(self):
        self.w = None
        self.b = None
        self.alphas = None
        self.sv_X = None
        self.sv_y = None

    def fit(self, X, y):
        """
        Train the SVM model using quadratic programming.
        """
        n_samples, n_features = X.shape


        y = y.astype(float).reshape(-1, 1)

        K = np.dot(X, X.T) * (y @ y.T)  # Kernel matrix
        P = matrix(K)
        q = matrix(-np.ones((n_samples, 1)))
        G = matrix(-np.eye(n_samples))
        h = matrix(np.zeros(n_samples))
        A = matrix(y.T)
        b = matrix(np.zeros(1))

        solvers.options['show_progress'] = False
        solution = solvers.qp(P, q, G, h, A, b)
        alphas = np.array(solution['x']).flatten()

        sv = alphas > 1e-5
        self.alphas = alphas[sv]
        self.sv_X = X[sv]
        self.sv_y = y[sv]

        # Compute weight vector (w)
        self.w = np.sum(self.alphas[:, None] * self.sv_y * self.sv_X, axis=0)

        # Compute bias term (b)
        self.b = np.mean(self.sv_y - np.dot(self.sv_X, self.w))

    def predict(self, X):
        """
        Predict labels for input samples.
        """
        return np.sign(np.dot(X, self.w) + self.b)


if __name__ == "__main__":
    # Training data
    X = np.array([[2, 3], [1, 1], [2, 1], [5, 6], [6, 6], [6, 7]])
    y = np.array([-1, -1, -1, 1, 1, 1])  # Class labels


    svm = SVM()
    svm.fit(X, y)


    X_test = np.array([[3, 3], [4, 5]])
    predictions = svm.predict(X_test)


    print("Test Samples:")
    for i, x in enumerate(X_test):
        print(f"Sample {i + 1}: {x}, Predicted Class: {predictions[i]}")

Test Samples:
Sample 1: [3 3], Predicted Class: -1.0
Sample 2: [4 5], Predicted Class: 1.0
